In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

# Computing Post-Optimality Sensitivities of a Constrained Optimization Problem

Lets consider a problem such that we have an active bound and an active inequality constraint.

\begin{align*}
\min_{\theta_0,\, \theta_1} \quad & f(\theta_0, \theta_1; \mathbf{p}) = (\theta_0 - p_0)^2 + \theta_0 \theta_1 + (\theta_1 + p_1)^2 - p_2 \\
\text{where} \quad \mathbf{p} &= \begin{bmatrix} 3 \\ 4 \\ 3 \end{bmatrix} \in \mathbb{R}^3 \\
\text{bounds:} \quad \theta_0 &\le 6 \\
\text{equality constraints:} \quad \theta_0 + \theta_1 &= 0
\end{align*}

We want to know the sensitivities of the optimization outputs with respect to the optimization inputs.

In this context, consider the outputs of the optimization to be the objective and any other functions of interest, $f$.

The design variables $\theta$ and Lagrange multipliers $\lambda$ are effectively the implicit outputs of the optimization.

The _inputs_ to the optimization process consists of:
- any independent parameters, $\bar{p}$
- the bounding values of any **active** design variables, $\bar{b}_{\theta}$
- the bounding values of any **active** constraints, $\bar{b}o_{g}$

In our case we have:

\begin{align*}
    \bar{p} &= \begin{bmatrix} p_0 \\ p_1 \\ p_2 \end{bmatrix} \\
    \bar{b}_{\theta} &= \begin{bmatrix} \theta_0^{ub} \end{bmatrix} \\
    \bar{b}_g &= \begin{bmatrix} g_0^{eq} \end{bmatrix}
\end{align*}

<!-- If active, we can treat the bound on $\theta_0$ as just another equality constraint.

\begin{align*}
  \bar{\mathcal{G}}(\bar{\theta}, \bar{p}) &= \begin{bmatrix}
                                   \theta_0 + \theta_1 \\
                                   \theta_0 - p_3
                                \end{bmatrix} = \bar 0
\end{align*}

**How will my system design ($\bar{\theta}^*$) respond to changes in my assumptions and system inputs ($\bar{p}$)?** -->

### The Universal Derivatives Equation

The UDE is:

\begin{align*}
  \left[ \frac{\partial \mathcal{R}}{\partial \mathcal{u}} \right] \left[ \frac{d u}{d \mathcal{R}} \right]
  &=
  \left[ I \right]
  =
  \left[ \frac{\partial \mathcal{R}}{\partial \mathcal{u}} \right]^T \left[ \frac{d u}{d \mathcal{R}} \right]^T\\
\end{align*}

Here, the residuals are the primal and dual residuals of the optimization process, given above.

## Applying the UDE to solving post-optimality sensitivities

In our case, the unknowns vector consists of
- the optimization parameters ($\bar{p}$)
- the bounding values of any active design variables ($\bar{b}_{\theta}$)
- the bounding values of any active constraints ($\bar{b}_{g}$)
- the design variables of the optimization ($\bar{\theta}$)
- the Lagrange multipliers associated with the active design variables ($\bar{\lambda}_{\theta}$)
- the Lagrange multipliers associated with the active constraints ($\bar{\lambda}_{g}$)
- the objective value **as well as** any other outputs for which we want the sensitivities ($f$)

The total size of the unknowns vector is $N_p + N_{\theta} + 2N_{\lambda \theta} + 2N_{\lambda g} + N_{f}$

\begin{align*}
  \hat{u} &=
  \begin{bmatrix}
    \bar{p} \\
    \bar{b}_{\theta} \\
    \bar{b}_{g} \\
    \bar{\theta} \\
    \bar{\lambda_{\theta}} \\
    \bar{\lambda_{g}} \\
    \bar{f}
  \end{bmatrix}
\end{align*}

Under the UDE, the corresponding residual equations for these unknowns are
- the implicit form of the parameter values
- the implicit form of the active design variable values
- the implicit form of the active constraint values
- the stationarity condition
- the active design variable residuals
- the active constraint residuals
- the implicit form of the explicit calculations of $f$

\begin{align*}
\bar{\mathcal{R}}
&=
\begin{bmatrix}
\bar{\mathcal{R}}_p \\
\bar{\mathcal{R}}_{b \theta} \\
\bar{\mathcal{R}}_{b g} \\
\bar{\mathcal{R}}_{\theta} \\
\bar{\mathcal{R}}_{\lambda \theta} \\
\bar{\mathcal{R}}_{\lambda g} \\
\bar{\mathcal{R}}_{f}
\end{bmatrix}
&=
\begin{bmatrix}
  \bar{p} - \check{p} \\[1.1ex]
  \hline \\
  \bar{b}_{\theta} - \check{b}_{\theta} \\[1.1ex]
  \hline \\
  \bar{b}_{g} - \check{b}_g \\[1.1ex]
  \hline \\
  \bar{r}_{\theta} - \left[ \nabla_{\bar{\theta}} \check{f} (\bar{\theta}, \bar{p}) + \nabla_{\bar{\theta}} \check{g}_{\mathcal{A}} (\bar{\theta}, \bar{p})^T \bar{\lambda}_g + \nabla_{\bar{\theta}} \check{\theta}_{\mathcal{A}} (\bar{\theta}, \bar{p})^T \bar{\lambda}_{\theta} \right] \\[1.1ex]
  \hline \\
  \bar{r}_{\lambda \theta} - \left[ \check{\theta}_{\mathcal{A}} \left( \bar{\theta} \right) - \bar{b}_{\theta} \right] \\[1.1ex]
  \hline \\
  \bar{r}_{\lambda g} - \left[ \check{g}_{\mathcal{A}} \left( \bar{\theta}, \bar{p} \right) - \bar{b}_g \right] \\[1.1ex]
  \hline \\
  \bar{f} - \check{f}\left(\bar{\theta}, \bar{p} \right) 
\end{bmatrix}
&= 
\begin{bmatrix}
  \bar{p} - \check{p} \\[1.1ex]
  \hline \\
  \bar{b}_{\theta} - \check{b}_{\theta} \\[1.1ex]
  \hline \\
  \bar{b}_{g} - \check{b}_g \\[1.1ex]
  \hline \\
  \bar{r}_{\theta} - \nabla_{\theta} \check{\mathcal{L}} \left( \bar{\theta}, \bar{p} \right) \\[1.1ex]
  \hline \\
  \bar{r}_{\lambda \theta} - \left[ \check{\theta}_{\mathcal{A}} \left( \bar{\theta} \right) - \bar{b}_{\theta} \right] \\[1.1ex]
  \hline \\
  \bar{r}_{\lambda g} - \left[ \check{g}_{\mathcal{A}} \left( \bar{\theta}, \bar{p} \right) - \bar{b}_g \right] \\[1.1ex]
  \hline \\
  \bar{f} - \check{f}\left(\bar{\theta}, \bar{p} \right) 
\end{bmatrix}
&=
\bar 0
\end{align*}

In order to find the total derivatives that we seek ($\frac{d f^*}{d \bar{p}}$ and $\frac{d \bar{\theta}^*}{d \bar{p}}$), we need $\frac{\partial \bar{\mathcal{R}}}{\partial \bar{u}}$.

The optimizer has served as the nonlinear solver in this case which has computed the values in the unknowns vector: $\bar{\theta}$, $\bar{\lambda}$, and $\bar{f}$ such that the residuals are satisfied.

\begin{align*}
\frac{\partial \bar{\mathcal{R}}}{\partial \bar{u}}
&=
\begin{bmatrix}
\frac{\partial \bar{\mathcal{R}_p}}{\partial \bar{p}} & 0 & 0 & 0 & 0 & 0 & 0 \\[1.1ex]
0 & \frac{\partial \bar{\mathcal{R}_{\bar{b} \theta}}}{\partial \bar{b}_{\theta}} & 0 & 0 & 0 & 0 & 0 \\[1.1ex]
0 & 0 & \frac{\partial \bar{\mathcal{R}_{\bar{b} g}}}{\partial \bar{b}_{g}} & 0 & 0 & 0 & 0 \\[1.1ex]
\frac{\partial \bar{\mathcal{R}_{\theta}}}{\partial \bar{p}} & 0 & 0 & \frac{\partial \bar{\mathcal{R}_{\theta}}}{\partial \bar{\theta}} & \frac{\partial \bar{\mathcal{R}_{\theta}}}{\partial \bar{\lambda_{\theta}}} & \frac{\partial \bar{\mathcal{R}_{\theta}}}{\partial \bar{\lambda_g}} & 0 \\[1.1ex]
0 & \frac{\partial \bar{\mathcal{R}_{\lambda \theta}}}{\partial \bar{b}_{\theta}} & 0 & \frac{\partial \bar{\mathcal{R}_{\lambda \theta}}}{\partial \bar{\theta}} & 0 & 0 & 0 \\[1.1ex]
\frac{\partial \bar{\mathcal{R}}_{\lambda g}}{\partial \bar{p}} & 0 & \frac{\partial \bar{\mathcal{R}}_{\lambda g}}{\partial \bar{b}_g} & \frac{\partial \bar{\mathcal{R}}_{\lambda g}}{\partial \bar{\theta}} & 0 & 0 & 0 \\[1.1ex]
\frac{\partial \bar{\mathcal{R}_f}}{\partial \bar{p}} & 0 & 0 & \frac{\partial \bar{\mathcal{R}_f}}{\partial \bar{\theta}} & 0 & 0 & \frac{\partial \bar{\mathcal{R}_f}}{\partial f}
\end{bmatrix}
&=
\begin{bmatrix}
\left[ I_p \right] & 0 & 0 & 0 & 0 & 0 & 0 \\[1.1ex]
0 & \left[ I_{b\theta} \right] & 0 & 0 & 0 & 0 & 0 \\[1.1ex]
0 & 0 & \left[ I_{bg} \right] & 0 & 0 & 0 & 0 \\[1.1ex]
-\frac{\partial \nabla_{\theta} \bar{\mathcal{L}}}{\partial \bar{p}} & 0 & 0 & - \nabla_{\theta}^2 \check{\mathcal{L}} & - \nabla_{\theta} \check{\theta}_{\mathcal{A}}^T & - \nabla_{\theta} \check{g}_{\mathcal{A}}^T & 0 \\[1.1ex]
0 & \left[ I_{b\theta} \right] & 0 & -\nabla_{\theta} \check{\theta}_{\mathcal{A}} & 0 & 0 & 0 \\[1.1ex]
-\frac{\partial \check{g}_{\mathcal{A}}}{\partial \bar{p}} & 0 & \left[ I_{bg} \right] & -\nabla_{\theta} \check{g}_{\mathcal{A}} & 0 & 0 & 0 \\[1.1ex]
-\frac{\partial \check{f}}{\partial \bar{p}} & 0 & 0 & -\frac{\partial \check{f}}{\partial \bar{\theta}} & 0 & 0 & \left[ I_f \right]
\end{bmatrix}
\end{align*}

This nomenclature can be a bit confusing.

**The _partial_ derivatives of the post-optimality residuals are the _total_ derivatives of the analysis.**

In this case of the stationarity residuals $\mathcal{R}_{\bar{\theta}}$, which already include _total_ derivatives of the analysis for the objective and constraint gradients, second derivatives are required.

The corresponding total derivaties which we need to solve for are:

\begin{align}
\frac{d \bar{u}}{d \bar{\mathcal{R}}}
&=
\begin{bmatrix}
  \frac{d \bar{p}}{d \bar{\mathcal{R}_p}} &
  \frac{d \bar{p}}{d \bar{\mathcal{R}_{b\theta}}} &
  \frac{d \bar{p}}{d \bar{\mathcal{R}_{bg}}} &
  \frac{d \bar{p}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{p}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{p}}{d \bar{\mathcal{R}_{\lambda g}}} &
  \frac{d \bar{p}}{d \bar{\mathcal{R}_f}}
\\[1.1ex]
  \frac{d \bar{b}_{\theta}}{d \bar{\mathcal{R}_p}} &
  \frac{d \bar{b}_{\theta}}{d \bar{\mathcal{R}_{b\theta}}} &
  \frac{d \bar{b}_{\theta}}{d \bar{\mathcal{R}_{bg}}} &
  \frac{d \bar{b}_{\theta}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{b}_{\theta}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{b}_{\theta}}{d \bar{\mathcal{R}_{\lambda g}}} &
  \frac{d \bar{b}_{\theta}}{d \bar{\mathcal{R}_f}}
\\[1.1ex]
  \frac{d \bar{b}_{g}}{d \bar{\mathcal{R}_p}} &
  \frac{d \bar{b}_{g}}{d \bar{\mathcal{R}_{b\theta}}} &
  \frac{d \bar{b}_{g}}{d \bar{\mathcal{R}_{bg}}} &
  \frac{d \bar{b}_{g}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{b}_{g}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{b}_{g}}{d \bar{\mathcal{R}_{\lambda g}}} &
  \frac{d \bar{b}_{g}}{d \bar{\mathcal{R}_f}}
\\[1.1ex]
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_p}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{b\theta}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{bg}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda g}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_f}}
\\[1.1ex]
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_p}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{b\theta}}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{bg}}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\lambda g}}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_f}}
\\[1.1ex]
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_p}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{b\theta}}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{bg}}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\lambda g}}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_f}}
\\[1.1ex]
  \frac{d \bar{f}}{d \bar{\mathcal{R}_p}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{b\theta}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{bg}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{\lambda g}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_f}}
\end{bmatrix}
&=
\begin{bmatrix}
  \left[ I_p \right] &
  0 &
  0 &
  0 &
  0 &
  0 &
  0
\\[1.1ex]
  0 &
  \left[ I_{b\theta} \right] &
  0 &
  0 &
  0 &
  0 &
  0
\\[1.1ex]
  0 &
  0 &
  \left[ I_{bg} \right] &
  0 &
  0 &
  0 &
  0
\\[1.1ex]
  \mathbf{\frac{d \bar{\theta}}{d \bar{p}}} &
  \mathbf{\frac{d \bar{\theta}}{d \bar{b_{\theta}}}} &
  \mathbf{\frac{d \bar{\theta}}{d \bar{b_g}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda g}}} &
  0
\\[1.1ex]
  \frac{d \bar{\lambda_{\theta}}}{d \bar{p}} &
  \frac{d \bar{\lambda_{\theta}}}{d \bar{b_{\theta}}} &
  \frac{d \bar{\lambda_{\theta}}}{d \bar{b_g}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\lambda g}}} &
  0
\\[1.1ex]
  \frac{d \bar{\lambda_{g}}}{d \bar{p}} &
  \frac{d \bar{\lambda_{g}}}{d \bar{b_{\theta}}} &
  \frac{d \bar{\lambda_{g}}}{d \bar{b_g}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\lambda g}}} &
  0
\\[1.1ex]
  \mathbf{\frac{d \bar{f}}{d \bar{p}}} &
  \mathbf{\frac{d \bar{f}}{d \bar{b_{\theta}}}} &
  \mathbf{\frac{d \bar{f}}{d \bar{b_g}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{\lambda g}}} &
  \left[ I_f \right]
\end{bmatrix}
\end{align}

The sensitivities of the objective and the design variable values with respect to the parameters of the optimization are highlighted.

In this case, we can solve them with four linear solves of the forward system, or three solves of the reverse system.

TODO: Need to explain how du/dRf becomes du/df.

The UDE for this case, in forward form, is

\begin{align*}
\begin{bmatrix}
\left[ I_p \right] & 0 & 0 & 0 & 0 & 0 & 0 \\[1.1ex]
0 & \left[ I_{b\theta} \right] & 0 & 0 & 0 & 0 & 0 \\[1.1ex]
0 & 0 & \left[ I_{bg} \right] & 0 & 0 & 0 & 0 \\[1.1ex]
-\frac{\partial \nabla_{\theta} \bar{\mathcal{L}}}{\partial \bar{p}} & 0 & 0 & - \nabla_{\theta}^2 \check{\mathcal{L}} & - \nabla_{\theta} \check{\theta}_{\mathcal{A}}^T & - \nabla_{\theta} \check{g}_{\mathcal{A}}^T & 0 \\[1.1ex]
0 & \left[ I_{b\theta} \right] & 0 & -\nabla_{\theta} \check{\theta}_{\mathcal{A}} & 0 & 0 & 0 \\[1.1ex]
-\frac{\partial \check{g}_{\mathcal{A}}}{\partial \bar{p}} & 0 & \left[ I_{bg} \right] & -\nabla_{\theta} \check{g}_{\mathcal{A}} & 0 & 0 & 0 \\[1.1ex]
-\frac{\partial \check{f}}{\partial \bar{p}} & 0 & 0 & -\frac{\partial \check{f}}{\partial \bar{\theta}} & 0 & 0 & \left[ I_f \right]
\end{bmatrix}
\begin{bmatrix}
  \left[ I_p \right] &
  0 &
  0 &
  0 &
  0 &
  0 &
  0
\\[1.1ex]
  0 &
  \left[ I_{b\theta} \right] &
  0 &
  0 &
  0 &
  0 &
  0
\\[1.1ex]
  0 &
  0 &
  \left[ I_{bg} \right] &
  0 &
  0 &
  0 &
  0
\\[1.1ex]
  \mathbf{\frac{d \bar{\theta}}{d \bar{p}}} &
  \mathbf{\frac{d \bar{\theta}}{d \bar{b_{\theta}}}} &
  \mathbf{\frac{d \bar{\theta}}{d \bar{b_g}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda g}}} &
  0
\\[1.1ex]
  \frac{d \bar{\lambda_{\theta}}}{d \bar{p}} &
  \frac{d \bar{\lambda_{\theta}}}{d \bar{b_{\theta}}} &
  \frac{d \bar{\lambda_{\theta}}}{d \bar{b_g}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\lambda g}}} &
  0
\\[1.1ex]
  \frac{d \bar{\lambda_{g}}}{d \bar{p}} &
  \frac{d \bar{\lambda_{g}}}{d \bar{b_{\theta}}} &
  \frac{d \bar{\lambda_{g}}}{d \bar{b_g}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\lambda g}}} &
  0
\\[1.1ex]
  \mathbf{\frac{d \bar{f}}{d \bar{p}}} &
  \mathbf{\frac{d \bar{f}}{d \bar{b_{\theta}}}} &
  \mathbf{\frac{d \bar{f}}{d \bar{b_g}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{\theta}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{\lambda \theta}}} &
  \frac{d \bar{f}}{d \bar{\mathcal{R}_{\lambda g}}} &
  \left[ I_f \right]
\end{bmatrix}
&=
\begin{bmatrix}
    \left[ I_p \right] & 0 & 0 & 0 & 0 & 0 & 0\\[1.1ex]
    0 & \left[ I_{b\theta} \right] & 0 & 0 & 0 & 0 & 0\\[1.1ex]
    0 & 0 & \left[ I_{bg} \right] & 0 & 0 & 0 & 0\\[1.1ex]
    0 & 0 & 0 & \left[ I_{\theta} \right] & 0 & 0 & 0\\[1.1ex]
    0 & 0 & 0 & 0 & \left[ I_{\lambda \theta} \right] & 0 & 0\\[1.1ex]
    0 & 0 & 0 & 0 & 0 & \left[ I_{\lambda g} \right] & 0\\[1.1ex]
    0 & 0 & 0 & 0 & 0 & 0 & \left[ I_f \right]
\end{bmatrix}
\end{align*}

The sensitivities of the objective and the design variable values with respect to the parameters of the optimization are highlighted.

The linear solve of this system can proceed one column of $\left[ \frac{d \bar{u}}{d \mathcal{R}} \right]$ at a time (the forward solve).
In this case we would need one solve for each column ($N_p + $N_{b\theta}$ + $N_b{g}$).
In our example optimization with three parameters, an active bound and an active equality constraint, this would be five solves.


Alternatively, we could transpose this system and solve it in reverse mode. Solving for one column at a time in the transposed system would mean solving once for each design variable and each output of interest. In our example optimziation this would be three solves.

\begin{align*}
\begin{bmatrix}
\left[ I_p \right]                                    & 0                    & 0                    & -\frac{\partial \nabla_{\theta} \bar{\mathcal{L}}}{\partial \bar{p}}^T & 0                    & -\frac{\partial \check{g}_{\mathcal{A}}}{\partial \bar{p}}^T & -\frac{\partial \check{f}}{\partial \bar{p}}^T \\[1.1ex]
0                                                     & \left[ I_{b\theta} \right] & 0                    & 0                                                                & \left[ I_{b\theta} \right] & 0                                                    & 0 \\[1.1ex]
0                                                     & 0                    & \left[ I_{bg} \right]      & 0                                                                & 0                    & \left[ I_{bg} \right]                                      & 0 \\[1.1ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta}^2 \check{\mathcal{L}}                        & -\nabla_{\theta} \check{\theta}_{\mathcal{A}}  & -\nabla_{\theta} \check{g}_{\mathcal{A}}                & -\frac{\partial \check{f}}{\partial \bar{\theta}}^T \\[1.1ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta} \check{\theta}_{\mathcal{A}}^T               & 0                    & 0                                                    & 0 \\[1.1ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta} \check{g}_{\mathcal{A}}^T                    & 0                    & 0                                                    & 0 \\[1.1ex]
0                                                     & 0                    & 0                    & 0                                                                & 0                    & 0                                                    & \left[ I_f \right]
\end{bmatrix}
\begin{bmatrix}
\left[ I_p \right]                                    & 0                         & 0                       & \mathbf{\frac{d \bar{\theta}}{d \bar{p}}}^T                    & \frac{d \bar{\lambda_{\theta}}}{d \bar{p}}^T              & \frac{d \bar{\lambda_{g}}}{d \bar{p}}^T                 & \mathbf{\frac{d \bar{f}}{d \bar{p}}}^T \\[1.1ex]
0                                                     & \left[ I_{b\theta} \right]      & 0                       & \mathbf{\frac{d \bar{\theta}}{d \bar{b_{\theta}}}}^T           & \frac{d \bar{\lambda_{\theta}}}{d \bar{b_{\theta}}}^T     & \frac{d \bar{\lambda_{g}}}{d \bar{b_{\theta}}}^T        & \mathbf{\frac{d \bar{f}}{d \bar{b_{\theta}}}}^T \\[1.1ex]
0                                                     & 0                         & \left[ I_{bg} \right]         & \mathbf{\frac{d \bar{\theta}}{d \bar{b_g}}}^T                  & \frac{d \bar{\lambda_{\theta}}}{d \bar{b_g}}^T            & \frac{d \bar{\lambda_{g}}}{d \bar{b_g}}^T                & \mathbf{\frac{d \bar{f}}{d \bar{b_g}}}^T \\[1.1ex]
0                                                     & 0                         & 0                       & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\theta}}}^T          & \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\theta}}}^T    & \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\theta}}}^T       & \frac{d \bar{f}}{d \bar{\mathcal{R}_{\theta}}}^T \\[1.1ex]
0                                                     & 0                         & 0                       & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda \theta}}}^T  & \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\lambda \theta}}}^T & \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\lambda \theta}}}^T & \frac{d \bar{f}}{d \bar{\mathcal{R}_{\lambda \theta}}}^T \\[1.1ex]
0                                                     & 0                         & 0                       & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda g}}}^T       & \frac{d \bar{\lambda \theta}}{d \bar{\mathcal{R}_{\lambda g}}}^T    & \frac{d \bar{\lambda g}}{d \bar{\mathcal{R}_{\lambda g}}}^T       & \frac{d \bar{f}}{d \bar{\mathcal{R}_{\lambda g}}}^T \\[1.1ex]
0                                                     & 0                         & 0                       & 0                                                                & 0                                                       & 0                                                         & \left[ I_f \right]
\end{bmatrix}
&=
\begin{bmatrix}
    \left[ I_p \right] & 0 & 0 & 0 & 0 & 0 & 0\\[1.1ex]
    0 & \left[ I_{b\theta} \right] & 0 & 0 & 0 & 0 & 0\\[1.1ex]
    0 & 0 & \left[ I_{bg} \right] & 0 & 0 & 0 & 0\\[1.1ex]
    0 & 0 & 0 & \left[ I_{\theta} \right] & 0 & 0 & 0\\[1.1ex]
    0 & 0 & 0 & 0 & \left[ I_{\lambda \theta} \right] & 0 & 0\\[1.1ex]
    0 & 0 & 0 & 0 & 0 & \left[ I_{\lambda g} \right] & 0\\[1.1ex]
    0 & 0 & 0 & 0 & 0 & 0 & \left[ I_f \right]
\end{bmatrix}
\end{align*}


## One more trick

Computing the Hessian of the Lagrangian is potentially expensive and we'd like to avoid it if possible.

When we seed this solve for the sensitivties of the objective (the last column) we have:

\begin{align*}
\begin{bmatrix}
\left[ I_p \right]                                    & 0                    & 0                    & -\frac{\partial \nabla_{\theta} \bar{\mathcal{L}}}{\partial \bar{p}}^T & 0                    & -\frac{\partial \check{g}_{\mathcal{A}}}{\partial \bar{p}}^T & -\frac{\partial \check{f}}{\partial \bar{p}}^T \\[1.3ex]
0                                                     & \left[ I_{b\theta} \right] & 0                    & 0                                                                & \left[ I_{b\theta} \right] & 0                                                    & 0 \\[1.3ex]
0                                                     & 0                    & \left[ I_{bg} \right]      & 0                                                                & 0                    & \left[ I_{bg} \right]                                      & 0 \\[1.3ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta}^2 \check{\mathcal{L}}                        & -\nabla_{\theta} \check{\theta}_{\mathcal{A}}  & -\nabla_{\theta} \check{g}_{\mathcal{A}}                & -\frac{\partial \check{f}}{\partial \bar{\theta}}^T \\[1.3ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta} \check{\theta}_{\mathcal{A}}^T               & 0                    & 0                                                    & 0 \\[1.3ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta} \check{g}_{\mathcal{A}}^T                    & 0                    & 0                                                    & 0 \\[1.3ex]
0                                                     & 0                    & 0                    & 0                                                                & 0                    & 0                                                    & \left[ I_f \right]
\end{bmatrix}
\begin{bmatrix}
\mathbf{\frac{d \bar{f}}{d \bar{p}}}^T \\[1.1ex]
\mathbf{\frac{d \bar{f}}{d \bar{b_{\theta}}}}^T \\[1.1ex]
\mathbf{\frac{d \bar{f}}{d \bar{b_g}}}^T \\[1.1ex]
\frac{d \bar{f}}{d \bar{\theta}}^T \\[1.1ex]
\frac{d \bar{f}}{d \bar{\lambda \theta}}^T \\[1.1ex]
\frac{d \bar{f}}{d \bar{\lambda g}}^T \\[1.1ex]
\left[ I_f \right]
\end{bmatrix}
&=
\begin{bmatrix}
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    \left[ I_f \right]
\end{bmatrix}
\end{align*}

When we seed this solve for the sensitivties of the design variables (the fourth column) we have:

\begin{align*}
\begin{bmatrix}
\left[ I_p \right]                                    & 0                    & 0                    & -\frac{\partial \nabla_{\theta} \bar{\mathcal{L}}}{\partial \bar{p}}^T & 0                    & -\frac{\partial \check{g}_{\mathcal{A}}}{\partial \bar{p}}^T & -\frac{\partial \check{f}}{\partial \bar{p}}^T \\[1.3ex]
0                                                     & \left[ I_{b\theta} \right] & 0                    & 0                                                                & \left[ I_{b\theta} \right] & 0                                                    & 0 \\[1.3ex]
0                                                     & 0                    & \left[ I_{bg} \right]      & 0                                                                & 0                    & \left[ I_{bg} \right]                                      & 0 \\[1.3ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta}^2 \check{\mathcal{L}}                        & -\nabla_{\theta} \check{\theta}_{\mathcal{A}}  & -\nabla_{\theta} \check{g}_{\mathcal{A}}                & -\frac{\partial \check{f}}{\partial \bar{\theta}}^T \\[1.3ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta} \check{\theta}_{\mathcal{A}}^T               & 0                    & 0                                                    & 0 \\[1.3ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta} \check{g}_{\mathcal{A}}^T                    & 0                    & 0                                                    & 0 \\[1.3ex]
0                                                     & 0                    & 0                    & 0                                                                & 0                    & 0                                                    & \left[ I_f \right]
\end{bmatrix}
\begin{bmatrix}
\mathbf{\frac{d \bar{\theta}}{d \bar{p}}}^T \\[1.1ex]
\mathbf{\frac{d \bar{\theta}}{d \bar{b_{\theta}}}}^T \\[1.1ex]
\mathbf{\frac{d \bar{\theta}}{d \bar{b_g}}}^T \\[1.1ex]
\frac{d \bar{\theta}}{d \bar{\theta}}^T \\[1.1ex]
\frac{d \bar{\theta}}{d \bar{\lambda \theta}}^T \\[1.1ex]
\frac{d \bar{\theta}}{d \bar{\lambda g}}^T \\[1.1ex]
\frac{d \bar{\theta}}{d \bar{f}}^T \\[1.1ex]
\end{bmatrix}
&=
\begin{bmatrix}
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    \left[ I_f \right]
\end{bmatrix}
\end{align*}

In this case, we seed the right vector with a one somewhere in the rows corresponding the $\theta$, while all other rows are zero.
This means that we need to multiply by the Hessian of the Lagrangian, which is expensive to compute.

We don't have this issue when we're solving for outputs, because the rows initially being multiplied by the Hessian of the Lagrangian are zero.

But if we had an output function that just echoed the values of the design variables, lets call it $f_{\bar{\theta}}$, then solving for the sensitivities of that function with respect to the parameters and bounds _would be the same thing_ as solving for the sensitivities of the design variables with respect to the parameters and bounds.

So we effectively augument the unknowns vector $\bar{u}$ with $f_{\bar{\theta}}$ and add its corresponding residual $\mathcal{R}_{f\theta} = \bar{f}_{\theta} - \bar{\theta}$

Instead of solving for the column of design variable sensitivities, we solve for additional columns corresponding to $f_{\theta}$.

\begin{align*}
\begin{bmatrix}
\left[ I_p \right]                                    & 0                    & 0                    & -\frac{\partial \nabla_{\theta} \bar{\mathcal{L}}}{\partial \bar{p}}^T & 0                    & -\frac{\partial \check{g}_{\mathcal{A}}}{\partial \bar{p}}^T & -\frac{\partial \check{f}}{\partial \bar{p}}^T & 0 \\[1.3ex]
0                                                     & \left[ I_{b\theta} \right] & 0                    & 0                                                                & \left[ I_{b\theta} \right] & 0                                                    & 0 & 0 \\[1.3ex]
0                                                     & 0                    & \left[ I_{bg} \right]      & 0                                                                & 0                    & \left[ I_{bg} \right]                                      & 0 & 0 \\[1.3ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta}^2 \check{\mathcal{L}}                        & -\nabla_{\theta} \check{\theta}_{\mathcal{A}}  & -\nabla_{\theta} \check{g}_{\mathcal{A}}                & -\frac{\partial \check{f}}{\partial \bar{\theta}}^T  & -\left[I_{f\theta}\right] \\[1.3ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta} \check{\theta}_{\mathcal{A}}^T               & 0                    & 0                                                    & 0 & 0\\[1.3ex]
0                                                     & 0                    & 0                    & - \nabla_{\theta} \check{g}_{\mathcal{A}}^T                    & 0                    & 0                                                    & 0 & 0 \\[1.3ex]
0                                                     & 0                    & 0                    & 0                                                                & 0                    & 0                                                    & \left[ I_f \right] & 0 \\[1.3ex]
0                                                     & 0                    & 0                    & 0                                                                & 0                    & 0                                                    & 0 & \left[ I_{f\theta} \right]
\end{bmatrix}
\begin{bmatrix}
\mathbf{\frac{d \bar{f}_{\theta}}{d \bar{p}}}^T \\[1.1ex]
\mathbf{\frac{d \bar{f}_{\theta}}{d \bar{b_{\theta}}}}^T \\[1.1ex]
\mathbf{\frac{d \bar{f}_{\theta}}{d \bar{b_g}}}^T \\[1.1ex]
\frac{d \bar{f}_{\theta}}{d \bar{\theta}}^T \\[1.1ex]
\frac{d \bar{f}_{\theta}}{d \bar{\lambda \theta}}^T \\[1.1ex]
\frac{d \bar{f}_{\theta}}{d \bar{\lambda g}}^T \\[1.1ex]
\frac{d \bar{f}_{\theta}}{d \bar{f}}^T \\[1.1ex]
\left[ I_{f\theta} \right]
\end{bmatrix}
&=
\begin{bmatrix}
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    0\\[1.8ex]
    \left[ I_{f\theta} \right]
\end{bmatrix}
\end{align*}

In [47]:
"""
Post-optimality sensitivity analysis using JAX and the UDE approach
with identity outputs to avoid Hessian computations.
"""

import jax
import jax.numpy as jnp
import numpy as np
from scipy.sparse.linalg import LinearOperator, gmres
from functools import partial

# Define the optimization problem functions
def f(Θ, p):
    """Objective function"""
    f_val = (Θ[0] - p[0])**2 + Θ[0] * Θ[1] + (Θ[1] + p[1])**2 - p[2]
    return jnp.array([f_val])

def Θ_active(Θ):
    """Active bound constraint on Θ[0]"""
    return jnp.array([Θ[0]])

def g_active(Θ, p):
    """Active equality constraint"""
    return jnp.array([Θ[0] + Θ[1]])

def compute_sensitivities_jax():
    """
    Compute post-optimality sensitivities using JAX and the UDE approach.
    """

    # Known optimal solution
    Θ_opt = jnp.array([6.0, -6.0])
    p_opt = jnp.array([3.0, 4.0, 3.0])
    b_theta_opt = jnp.array([6.0])
    b_g_opt = jnp.array([0.0])
    λ_theta_opt = jnp.array([2.0])
    λ_g_opt = jnp.array([-2.0])

    # Problem dimensions
    n_p = 3  # parameters
    n_btheta = 1  # active bounds on design vars
    n_bg = 1  # active constraint bounds
    n_theta = 2  # design variables
    n_lambda_theta = 1  # multipliers for active bounds
    n_lambda_g = 1  # multipliers for active constraints
    n_f = 1  # original objective
    n_f_theta = 2  # identity outputs for design variables
    n_outputs_total = n_f + n_f_theta  # total outputs

    total_size = n_p + n_btheta + n_bg + n_theta + n_lambda_theta + n_lambda_g + n_outputs_total

    # Compute required derivatives using JAX

    # Objective derivatives
    df_dtheta = jax.jacobian(f, argnums=0)(Θ_opt, p_opt).reshape(-1)
    df_dp = jax.jacobian(f, argnums=1)(Θ_opt, p_opt).reshape(-1)

    # Active bound derivatives (Θ[0] - b_theta = 0)
    dtheta_active_dtheta = jax.jacobian(Θ_active, argnums=0)(Θ_opt).reshape(n_lambda_theta, n_theta)

    # Active constraint derivatives
    dg_active_dtheta = jax.jacobian(g_active, argnums=0)(Θ_opt, p_opt).reshape(n_lambda_g, n_theta)
    dg_active_dp = jax.jacobian(g_active, argnums=1)(Θ_opt, p_opt).reshape(n_lambda_g, n_p)

    # Lagrangian gradient: ∇L = ∇f + λ_theta^T ∇Θ_active + λ_g^T ∇g_active
    def lagrangian(Θ, p, λ_theta, λ_g):
        return f(Θ, p)[0] + λ_theta @ Θ_active(Θ) + λ_g @ g_active(Θ, p)

    # Hessian of Lagrangian w.r.t. Θ
    d2L_dtheta2 = jax.hessian(lagrangian, argnums=0)(Θ_opt, p_opt, λ_theta_opt, λ_g_opt)

    # Mixed derivatives of Lagrangian
    d2L_dtheta_dp = jax.jacobian(jax.grad(lagrangian, argnums=0), argnums=1)(
        Θ_opt, p_opt, λ_theta_opt, λ_g_opt
    )

    def matvec_transpose(v):
        """
        Compute matrix-vector product for [∂R/∂u]^T @ v
        This implements the transpose of the UDE Jacobian matrix.
        """
        # Split v into blocks
        idx = 0
        v_p = v[idx:idx+n_p]
        idx += n_p
        v_btheta = v[idx:idx+n_btheta]
        idx += n_btheta
        v_bg = v[idx:idx+n_bg]
        idx += n_bg
        v_theta = v[idx:idx+n_theta]
        idx += n_theta
        v_lambda_theta = v[idx:idx+n_lambda_theta]
        idx += n_lambda_theta
        v_lambda_g = v[idx:idx+n_lambda_g]
        idx += n_lambda_g
        v_outputs = v[idx:idx+n_outputs_total]
        v_f = v_outputs[:n_f]
        v_f_theta = v_outputs[n_f:]

        print(v)

        # Initialize result
        result = np.zeros(total_size)

        # Block 1: Effect on p
        result[:n_p] = v_p
        if np.any(v_theta):
            result[:n_p] -= d2L_dtheta_dp.T @ v_theta
        if np.any(v_lambda_g):
            result[:n_p] -= dg_active_dp.T @ v_lambda_g
        if np.any(v_f):
            result[:n_p] -= df_dp * v_f[0]

        # Block 2: Effect on b_theta
        idx = n_p
        result[idx:idx+n_btheta] = v_btheta + v_lambda_theta

        # Block 3: Effect on b_g
        idx += n_btheta
        result[idx:idx+n_bg] = v_bg + v_lambda_g

        # Block 4: Effect on theta
        idx += n_bg
        result_theta = np.zeros(n_theta)
        if np.any(v_theta):
            print('using hessian!')
            result_theta -= d2L_dtheta2 @ v_theta
        if np.any(v_lambda_theta):
            result_theta -= dtheta_active_dtheta.T @ v_lambda_theta
        if np.any(v_lambda_g):
            result_theta -= dg_active_dtheta.T @ v_lambda_g
        if np.any(v_f):
            result_theta -= df_dtheta * v_f[0]
        # Identity outputs contribution (avoids Hessian!)
        if np.any(v_f_theta):
            result_theta -= v_f_theta  # -I @ v_f_theta
        result[idx:idx+n_theta] = result_theta

        # Block 5: Effect on lambda_theta
        idx += n_theta
        if np.any(v_theta):
            result[idx:idx+n_lambda_theta] = -dtheta_active_dtheta @ v_theta

        # Block 6: Effect on lambda_g
        idx += n_lambda_theta
        if np.any(v_theta):
            result[idx:idx+n_lambda_g] = -dg_active_dtheta @ v_theta

        # Block 7: Effect on outputs
        idx += n_lambda_g
        result[idx:idx+n_outputs_total] = v_outputs

        return result

    # Create LinearOperator
    A_transpose = LinearOperator((total_size, total_size), matvec=matvec_transpose)

    # Storage for sensitivities
    sensitivities = {}

    # Solve for each output
    output_names = ['f', 'θ₀', 'θ₁']

    for i, name in enumerate(output_names):
        # Create RHS vector with 1 in appropriate output position
        rhs = np.zeros(total_size)
        output_start_idx = n_p + n_btheta + n_bg + n_theta + n_lambda_theta + n_lambda_g
        rhs[output_start_idx + i] = 1.0

        # Solve the system
        print(f"\nSolving for sensitivities of {name}...")
        solution, info = gmres(A_transpose, rhs, rtol=1e-10, maxiter=1000)

        if info == 0:
            # Extract sensitivities
            sens_p = solution[:n_p]
            sens_btheta = solution[n_p:n_p+n_btheta]
            sens_bg = solution[n_p+n_btheta:n_p+n_btheta+n_bg]

            sensitivities[name] = {
                'wrt_p': sens_p,
                'wrt_btheta': sens_btheta,
                'wrt_bg': sens_bg
            }

            print(f"  d{name}/dp₀ = {sens_p[0]:+.6f}")
            print(f"  d{name}/dp₁ = {sens_p[1]:+.6f}")
            print(f"  d{name}/dp₂ = {sens_p[2]:+.6f}")
            print(f"  d{name}/db_θ₀ = {sens_btheta[0]:+.6f}")
            print(f"  d{name}/db_g₀ = {sens_bg[0]:+.6f}")
        else:
            print(f"  Warning: GMRES did not converge (info={info})")

    return sensitivities

def verify_with_finite_differences(sensitivities, h=1e-6):
    """
    Verify sensitivities using finite differences.
    """
    print("\n" + "="*60)
    print("Verification with Finite Differences")
    print("="*60)

    # Base values
    Θ_base = jnp.array([6.0, -6.0])
    p_base = jnp.array([3.0, 4.0, 3.0])
    b_theta_base = 6.0

    # For simplicity, we'll verify df/dp₀ and dθ₀/db_θ₀

    # Verify df/dp₀
    def solve_optimization(p_val, b_theta_val):
        """
        Simplified optimization solve assuming active constraints remain active.
        For the active set: θ₀ = b_theta_val, θ₁ = -b_theta_val
        """
        θ_opt = jnp.array([b_theta_val, -b_theta_val])
        f_val = f(θ_opt, p_val)[0]
        return f_val, θ_opt

    # df/dp₀ by finite differences
    p_plus = p_base.at[0].set(p_base[0] + h)
    f_plus, _ = solve_optimization(p_plus, b_theta_base)
    p_minus = p_base.at[0].set(p_base[0] - h)
    f_minus, _ = solve_optimization(p_minus, b_theta_base)
    df_dp0_fd = (f_plus - f_minus) / (2 * h)

    print(f"\ndf/dp₀:")
    print(f"  UDE:    {sensitivities['f']['wrt_p'][0]:+.6f}")
    print(f"  FD:     {df_dp0_fd:+.6f}")
    print(f"  Error:  {abs(sensitivities['f']['wrt_p'][0] - df_dp0_fd):.2e}")

    # dθ₀/db_θ₀ by finite differences (should be 1.0 since θ₀ = b_θ₀ at optimum)
    _, theta_plus = solve_optimization(p_base, b_theta_base + h)
    _, theta_minus = solve_optimization(p_base, b_theta_base - h)
    dtheta0_dbtheta_fd = (theta_plus[0] - theta_minus[0]) / (2 * h)

    print(f"\ndθ₀/db_θ₀:")
    print(f"  UDE:    {sensitivities['θ₀']['wrt_btheta'][0]:+.6f}")
    print(f"  FD:     {dtheta0_dbtheta_fd:+.6f}")
    print(f"  Error:  {abs(sensitivities['θ₀']['wrt_btheta'][0] - dtheta0_dbtheta_fd):.2e}")

if __name__ == "__main__":
    print("="*60)
    print("Post-Optimality Sensitivity Analysis using JAX")
    print("="*60)

    # Compute sensitivities
    sensitivities = compute_sensitivities_jax()

    # Verify with finite differences
    verify_with_finite_differences(sensitivities)

    print("\n" + "="*60)
    print("Summary of Key Results")
    print("="*60)
    print("\nNote: Since θ₀ is bounded at 6.0 and the constraint θ₀ + θ₁ = 0 is active:")
    print("- Changes in b_θ₀ directly affect θ₀ (dθ₀/db_θ₀ = 1)")
    print("- Changes in b_θ₀ inversely affect θ₁ (dθ₁/db_θ₀ = -1)")
    print("- The objective is most sensitive to p₀ (df/dp₀ = -6)")

Post-Optimality Sensitivity Analysis using JAX
[0 0 0 0 0 0 0 0 0 0 0 0]

Solving for sensitivities of f...
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[ 0.79471941  0.52981294  0.13245324  0.          0.          0.
 -0.26490647  0.          0.          0.          0.          0.        ]
using hessian!
[-0.05448526  0.48128647 -0.00908088  0.          0.          0.25880499
  0.79457673  0.          0.25880499  0.          0.          0.        ]
using hessian!
[ 0.0358737   0.1143094  -0.08025979  0.          0.25871623 -0.81719063
  0.29611    -0.25871623 -0.29975817  0.          0.          0.        ]
using hessian!
[-0.20882503  0.26031995 -0.00167372 -0.35466222  0.60993311  0.17641056
 -0.10667204  0.45405356 -0.37703434  0.          0.          0.        ]
using hessian!
[-0.28802544  0.28758436  0.09178881  0.72515809  0.3594584   0.13373555
 -0.2430132  -0.30577893  0.02013532  0.          0.          0.        ]
using hessian!
[-6.00000000e+00 -4.00000000e+00 -1.00000000e+00 -2.

Heres a version without the hessian

In [62]:
"""
Post-optimality sensitivity analysis using JAX and the UDE approach
with identity outputs to avoid Hessian computations.
"""

import jax
import jax.numpy as jnp
import numpy as np
from scipy.sparse.linalg import LinearOperator, gmres
from functools import partial

# Define the optimization problem functions
def f(Θ, p):
    """Objective function"""
    f_val = (Θ[0] - p[0])**2 + Θ[0] * Θ[1] + (Θ[1] + p[1])**2 - p[2]
    return jnp.array([f_val])

def Θ_active(Θ):
    """Active bound constraint on Θ[0]"""
    return jnp.array([Θ[0]])

def g_active(Θ, p):
    """Active equality constraint"""
    return jnp.array([Θ[0] + Θ[1]])

def compute_sensitivities_jax():
    """
    Compute post-optimality sensitivities using JAX and the UDE approach.
    """

    # Known optimal solution
    Θ_opt = jnp.array([6.0, -6.0])
    p_opt = jnp.array([3.0, 4.0, 3.0])
    b_theta_opt = jnp.array([6.0])
    b_g_opt = jnp.array([0.0])
    λ_theta_opt = jnp.array([2.0])
    λ_g_opt = jnp.array([-2.0])

    # Problem dimensions
    n_p = 3  # parameters
    n_btheta = 1  # active bounds on design vars
    n_bg = 1  # active constraint bounds
    n_theta = 2  # design variables
    n_lambda_theta = 1  # multipliers for active bounds
    n_lambda_g = 1  # multipliers for active constraints
    n_f = 1  # original objective
    n_f_theta = 2  # identity outputs for design variables
    n_outputs_total = n_f + n_f_theta  # total outputs

    total_size = n_p + n_btheta + n_bg + n_theta + n_lambda_theta + n_lambda_g + n_outputs_total

    # Compute required derivatives using JAX (NO HESSIAN NEEDED!)

    # Objective derivatives
    df_dtheta = jax.jacobian(f, argnums=0)(Θ_opt, p_opt).reshape(-1)
    df_dp = jax.jacobian(f, argnums=1)(Θ_opt, p_opt).reshape(-1)

    # Active bound derivatives (Θ[0] - b_theta = 0)
    dtheta_active_dtheta = jax.jacobian(Θ_active, argnums=0)(Θ_opt).reshape(n_lambda_theta, n_theta)

    # Active constraint derivatives
    dg_active_dtheta = jax.jacobian(g_active, argnums=0)(Θ_opt, p_opt).reshape(n_lambda_g, n_theta)
    dg_active_dp = jax.jacobian(g_active, argnums=1)(Θ_opt, p_opt).reshape(n_lambda_g, n_p)

    # Lagrangian gradient: ∇L = ∇f + λ_theta^T ∇Θ_active + λ_g^T ∇g_active
    def lagrangian(Θ, p, λ_theta, λ_g):
        return f(Θ, p)[0] + λ_theta @ Θ_active(Θ) + λ_g @ g_active(Θ, p)

    # Mixed derivatives of Lagrangian (still needed for the p block)
    d2L_dtheta_dp = jax.jacobian(jax.grad(lagrangian, argnums=0), argnums=1)(
        Θ_opt, p_opt, λ_theta_opt, λ_g_opt
    )

    def matvec_transpose(v):
        """
        Compute matrix-vector product for [∂R/∂u]^T @ v
        This implements the transpose of the UDE Jacobian matrix.

        KEY: When v corresponds to an identity output seed (v_f_theta nonzero),
        we avoid the Hessian computation entirely!
        """
        # Split v into blocks
        idx = 0
        v_p = v[idx:idx+n_p]
        idx += n_p
        v_btheta = v[idx:idx+n_btheta]
        idx += n_btheta
        v_bg = v[idx:idx+n_bg]
        idx += n_bg
        v_theta = v[idx:idx+n_theta]
        idx += n_theta
        v_lambda_theta = v[idx:idx+n_lambda_theta]
        idx += n_lambda_theta
        v_lambda_g = v[idx:idx+n_lambda_g]
        idx += n_lambda_g
        v_outputs = v[idx:idx+n_outputs_total]
        v_f = v_outputs[:n_f]
        v_f_theta = v_outputs[n_f:]

        # Initialize result
        result = np.zeros(total_size)

        # Block 1: Effect on p
        result[:n_p] = v_p
        if np.any(v_theta):
            result[:n_p] -= d2L_dtheta_dp.T @ v_theta
        if np.any(v_lambda_g):
            result[:n_p] -= dg_active_dp.T @ v_lambda_g
        if np.any(v_f):
            result[:n_p] -= df_dp * v_f[0]

        # Block 2: Effect on b_theta
        idx = n_p
        result[idx:idx+n_btheta] = v_btheta + v_lambda_theta

        # Block 3: Effect on b_g
        idx += n_btheta
        result[idx:idx+n_bg] = v_bg + v_lambda_g

        # Block 4: Effect on theta
        idx += n_bg
        result_theta = np.zeros(n_theta)

        # CRITICAL: Only compute Hessian term if v_theta is nonzero
        # When solving for θ sensitivities via identity outputs, v_theta = 0!
        # if np.any(v_theta):
        #     # We would need Hessian here, but for identity output solves this is zero!
        #     raise ValueError("Hessian computation required - this shouldn't happen for identity outputs!")

        # print(f'{v_p=}')
        # print(f'{v_btheta=}')
        # print(f'{v_bg=}')
        print(f'{v_theta=}')
        # print(f'{v_lambda_theta=}')
        # print(f'{v_lambda_g=}')
        # print(f'{v_outputs=}')
        # print(f'{v_f=}')
        print(f'{v_f_theta=}')
        # print()


        if np.any(v_lambda_theta):
            result_theta -= dtheta_active_dtheta.T @ v_lambda_theta
        if np.any(v_lambda_g):
            result_theta -= dg_active_dtheta.T @ v_lambda_g
        if np.any(v_f):
            result_theta -= df_dtheta * v_f[0]

        # Identity outputs contribution (avoids Hessian!)
        if np.any(v_f_theta):
            result_theta -= v_f_theta  # -I @ v_f_theta

        result[idx:idx+n_theta] = result_theta

        # Block 5: Effect on lambda_theta
        idx += n_theta
        if np.any(v_theta):
            result[idx:idx+n_lambda_theta] = -dtheta_active_dtheta @ v_theta

        # Block 6: Effect on lambda_g
        idx += n_lambda_theta
        if np.any(v_theta):
            result[idx:idx+n_lambda_g] = -dg_active_dtheta @ v_theta

        # Block 7: Effect on outputs
        idx += n_lambda_g
        result[idx:idx+n_outputs_total] = v_outputs

        return result

    # Create LinearOperator
    A_transpose = LinearOperator((total_size, total_size), matvec=matvec_transpose)

    # Storage for sensitivities
    sensitivities = {}

    # Solve for each output
    output_names = ['f', 'θ₀', 'θ₁']

    print("\nNOTE: Computing sensitivities WITHOUT computing the Hessian!")
    print("The identity output approach allows us to skip ∇²L entirely.\n")

    for i, name in enumerate(output_names):
        # Create RHS vector with 1 in appropriate output position
        rhs = np.zeros(total_size)
        output_start_idx = n_p + n_btheta + n_bg + n_theta + n_lambda_theta + n_lambda_g
        rhs[output_start_idx + i] = 1.0

        # Check that we're not triggering Hessian computation
        # For identity outputs (i > 0), v_theta should always be zero in the iteration
        is_identity_output = (i > 0)

        # Solve the system
        print(f"\nSolving for sensitivities of {name}...")
        if is_identity_output:
            print(f"  (Using identity output - no Hessian needed!)")

        solution, info = gmres(A_transpose, rhs, rtol=1e-10, maxiter=1000)
        print(solution)

        if info == 0:
            # Extract sensitivities
            sens_p = solution[:n_p]
            sens_btheta = solution[n_p:n_p+n_btheta]
            sens_bg = solution[n_p+n_btheta:n_p+n_btheta+n_bg]

            sensitivities[name] = {
                'wrt_p': sens_p,
                'wrt_btheta': sens_btheta,
                'wrt_bg': sens_bg
            }

            print(f"  d{name}/dp₀ = {sens_p[0]:+.6f}")
            print(f"  d{name}/dp₁ = {sens_p[1]:+.6f}")
            print(f"  d{name}/dp₂ = {sens_p[2]:+.6f}")
            print(f"  d{name}/db_θ₀ = {sens_btheta[0]:+.6f}")
            print(f"  d{name}/db_g₀ = {sens_bg[0]:+.6f}")
        else:
            print(f"  Warning: GMRES did not converge (info={info})")

    return sensitivities

def verify_with_finite_differences(sensitivities, h=1e-6):
    """
    Verify sensitivities using finite differences.
    """
    print("\n" + "="*60)
    print("Verification with Finite Differences")
    print("="*60)

    # Base values
    Θ_base = jnp.array([6.0, -6.0])
    p_base = jnp.array([3.0, 4.0, 3.0])
    b_theta_base = 6.0

    # For simplicity, we'll verify df/dp₀ and dθ₀/db_θ₀

    def solve_optimization(p_val, b_theta_val):
        """
        Simplified optimization solve assuming active constraints remain active.
        For the active set: θ₀ = b_theta_val, θ₁ = -b_theta_val
        """
        θ_opt = jnp.array([b_theta_val, -b_theta_val])
        f_val = f(θ_opt, p_val)[0]
        return f_val, θ_opt

    # df/dp₀ by finite differences
    p_plus = p_base.at[0].set(p_base[0] + h)
    f_plus, _ = solve_optimization(p_plus, b_theta_base)
    p_minus = p_base.at[0].set(p_base[0] - h)
    f_minus, _ = solve_optimization(p_minus, b_theta_base)
    df_dp0_fd = (f_plus - f_minus) / (2 * h)

    print(f"\ndf/dp₀:")
    print(f"  UDE:    {sensitivities['f']['wrt_p'][0]:+.6f}")
    print(f"  FD:     {df_dp0_fd:+.6f}")
    print(f"  Error:  {abs(sensitivities['f']['wrt_p'][0] - df_dp0_fd):.2e}")

    # dθ₀/db_θ₀ by finite differences (should be 1.0 since θ₀ = b_θ₀ at optimum)
    _, theta_plus = solve_optimization(p_base, b_theta_base + h)
    _, theta_minus = solve_optimization(p_base, b_theta_base - h)
    dtheta0_dbtheta_fd = (theta_plus[0] - theta_minus[0]) / (2 * h)

    print(f"\ndθ₀/db_θ₀:")
    print(f"  UDE:    {sensitivities['θ₀']['wrt_btheta'][0]:+.6f}")
    print(f"  FD:     {dtheta0_dbtheta_fd:+.6f}")
    print(f"  Error:  {abs(sensitivities['θ₀']['wrt_btheta'][0] - dtheta0_dbtheta_fd):.2e}")

    # Additional verification: dθ₁/db_θ₀ (should be -1.0 due to equality constraint)
    dtheta1_dbtheta_fd = (theta_plus[1] - theta_minus[1]) / (2 * h)

    print(f"\ndθ₁/db_θ₀:")
    print(f"  UDE:    {sensitivities['θ₁']['wrt_btheta'][0]:+.6f}")
    print(f"  FD:     {dtheta1_dbtheta_fd:+.6f}")
    print(f"  Error:  {abs(sensitivities['θ₁']['wrt_btheta'][0] - dtheta1_dbtheta_fd):.2e}")

if __name__ == "__main__":
    print("="*60)
    print("Post-Optimality Sensitivity Analysis using JAX")
    print("WITHOUT Computing the Hessian of the Lagrangian")
    print("="*60)

    # Compute sensitivities
    sensitivities = compute_sensitivities_jax()

    # Verify with finite differences
    verify_with_finite_differences(sensitivities)

    print("\n" + "="*60)
    print("Summary")
    print("="*60)
    print("\nKey insight: By using identity outputs for θ₀ and θ₁,")
    print("we completely avoid computing the expensive Hessian ∇²L.")
    print("\nThe trick is that when solving for θ sensitivities,")
    print("the RHS vector has zeros in the θ residual positions,")
    print("so v_theta = 0 throughout the GMRES iteration,")
    print("eliminating the need for Hessian-vector products!")

Post-Optimality Sensitivity Analysis using JAX
WITHOUT Computing the Hessian of the Lagrangian
v_theta=array([0, 0], dtype=int8)
v_f_theta=array([0, 0], dtype=int8)

NOTE: Computing sensitivities WITHOUT computing the Hessian!
The identity output approach allows us to skip ∇²L entirely.


Solving for sensitivities of f...
v_theta=array([0., 0.])
v_f_theta=array([0., 0.])
v_theta=array([ 0.        , -0.26490647])
v_f_theta=array([0., 0.])
v_theta=array([0.        , 0.52245748])
v_f_theta=array([0., 0.])
v_theta=array([-0.5754727 , -0.24824313])
v_f_theta=array([0., 0.])
v_theta=array([ 0.42731203, -0.55680053])
v_f_theta=array([0., 0.])
v_theta=array([-0.19323376, -0.30288722])
v_f_theta=array([0., 0.])
v_theta=array([-6.66133815e-16,  0.00000000e+00])
v_f_theta=array([0., 0.])
[-6.00000000e+00 -4.00000000e+00 -1.00000000e+00 -2.00000000e+00
  2.00000000e+00 -6.66133815e-16  0.00000000e+00  2.00000000e+00
 -2.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
  df/dp₀ = -6.00

## The All-JVP Way

In [63]:
"""
Post-optimality sensitivity analysis using JAX and the UDE approach
with all matrix-vector products computed using Jacobian-vector products.
"""

import jax
import jax.numpy as jnp
import numpy as np
from scipy.sparse.linalg import LinearOperator, gmres
from functools import partial

# Define the optimization problem functions
def f(Θ, p):
    """Objective function"""
    f_val = (Θ[0] - p[0])**2 + Θ[0] * Θ[1] + (Θ[1] + p[1])**2 - p[2]
    return jnp.array([f_val])

def Θ_active(Θ):
    """Active bound constraint on Θ[0]"""
    return jnp.array([Θ[0]])

def g_active(Θ, p):
    """Active equality constraint"""
    return jnp.array([Θ[0] + Θ[1]])

def compute_sensitivities_jax():
    """
    Compute post-optimality sensitivities using JAX and the UDE approach.
    Uses Jacobian-vector products for all matrix-vector operations.
    """

    # Known optimal solution
    Θ_opt = jnp.array([6.0, -6.0])
    p_opt = jnp.array([3.0, 4.0, 3.0])
    b_theta_opt = jnp.array([6.0])
    b_g_opt = jnp.array([0.0])
    λ_theta_opt = jnp.array([2.0])
    λ_g_opt = jnp.array([-2.0])

    # Problem dimensions
    n_p = 3  # parameters
    n_btheta = 1  # active bounds on design vars
    n_bg = 1  # active constraint bounds
    n_theta = 2  # design variables
    n_lambda_theta = 1  # multipliers for active bounds
    n_lambda_g = 1  # multipliers for active constraints
    n_f = 1  # original objective
    n_f_theta = 2  # identity outputs for design variables
    n_outputs_total = n_f + n_f_theta  # total outputs

    total_size = n_p + n_btheta + n_bg + n_theta + n_lambda_theta + n_lambda_g + n_outputs_total

    # Define Lagrangian and its gradient
    def lagrangian(Θ, p, λ_theta, λ_g):
        """Lagrangian function"""
        return f(Θ, p)[0] + λ_theta @ Θ_active(Θ) + λ_g @ g_active(Θ, p)

    def lagrangian_grad(Θ, p, λ_theta, λ_g):
        """Gradient of Lagrangian w.r.t. Θ"""
        return jax.grad(lagrangian, argnums=0)(Θ, p, λ_theta, λ_g)

    # Function to compute Hessian-vector product using finite differences
    def hessian_vector_product(v_theta, h=1e-8):
        """
        Compute H @ v where H is the Hessian of the Lagrangian w.r.t. Θ
        Uses finite differences: H @ v ≈ (∇L(Θ + h*v) - ∇L(Θ - h*v)) / (2*h)
        """
        if not np.any(v_theta):
            return np.zeros(n_theta)

        v_theta_jax = jnp.array(v_theta)

        # Compute gradients at perturbed points
        grad_plus = lagrangian_grad(Θ_opt + h * v_theta_jax, p_opt, λ_theta_opt, λ_g_opt)
        grad_minus = lagrangian_grad(Θ_opt - h * v_theta_jax, p_opt, λ_theta_opt, λ_g_opt)

        # Finite difference approximation of Hessian-vector product
        hvp = (grad_plus - grad_minus) / (2 * h)

        return np.array(hvp)

    # JVP functions for matrix-transpose-vector products
    def jvp_d2L_dtheta_dp_T(v_theta):
        """
        Compute [∂²L/∂θ∂p]^T @ v_theta using JVP
        This is equivalent to ∂/∂p [∇_θ L^T @ v_theta]
        """
        if not np.any(v_theta):
            return np.zeros(n_p)

        v_theta_jax = jnp.array(v_theta)

        def scalar_func(p):
            grad_L = lagrangian_grad(Θ_opt, p, λ_theta_opt, λ_g_opt)
            return jnp.dot(grad_L, v_theta_jax)

        result = jax.grad(scalar_func)(p_opt)
        return np.array(result)

    def jvp_dg_active_dp_T(v_lambda_g):
        """
        Compute [∂g_active/∂p]^T @ v_lambda_g using JVP
        This is equivalent to ∂/∂p [g_active^T @ v_lambda_g]
        """
        if not np.any(v_lambda_g):
            return np.zeros(n_p)

        v_lambda_g_jax = jnp.array(v_lambda_g)

        def scalar_func(p):
            g_vals = g_active(Θ_opt, p)
            return jnp.dot(g_vals, v_lambda_g_jax)

        result = jax.grad(scalar_func)(p_opt)
        return np.array(result)

    def jvp_df_dp_T(v_f):
        """
        Compute [∂f/∂p]^T @ v_f using JVP
        This is equivalent to ∂/∂p [f^T @ v_f]
        """
        if not np.any(v_f):
            return np.zeros(n_p)

        v_f_jax = jnp.array(v_f)

        def scalar_func(p):
            f_vals = f(Θ_opt, p)
            return jnp.dot(f_vals, v_f_jax)

        result = jax.grad(scalar_func)(p_opt)
        return np.array(result)

    def jvp_dtheta_active_dtheta_T(v_lambda_theta):
        """
        Compute [∂Θ_active/∂θ]^T @ v_lambda_theta using JVP
        This is equivalent to ∂/∂θ [Θ_active^T @ v_lambda_theta]
        """
        if not np.any(v_lambda_theta):
            return np.zeros(n_theta)

        v_lambda_theta_jax = jnp.array(v_lambda_theta)

        def scalar_func(theta):
            theta_active_vals = Θ_active(theta)
            return jnp.dot(theta_active_vals, v_lambda_theta_jax)

        result = jax.grad(scalar_func)(Θ_opt)
        return np.array(result)

    def jvp_dg_active_dtheta_T(v_lambda_g):
        """
        Compute [∂g_active/∂θ]^T @ v_lambda_g using JVP
        This is equivalent to ∂/∂θ [g_active^T @ v_lambda_g]
        """
        if not np.any(v_lambda_g):
            return np.zeros(n_theta)

        v_lambda_g_jax = jnp.array(v_lambda_g)

        def scalar_func(theta):
            g_vals = g_active(theta, p_opt)
            return jnp.dot(g_vals, v_lambda_g_jax)

        result = jax.grad(scalar_func)(Θ_opt)
        return np.array(result)

    def jvp_df_dtheta_T(v_f):
        """
        Compute [∂f/∂θ]^T @ v_f using JVP
        This is equivalent to ∂/∂θ [f^T @ v_f]
        """
        if not np.any(v_f):
            return np.zeros(n_theta)

        v_f_jax = jnp.array(v_f)

        def scalar_func(theta):
            f_vals = f(theta, p_opt)
            return jnp.dot(f_vals, v_f_jax)

        result = jax.grad(scalar_func)(Θ_opt)
        return np.array(result)

    # Regular JVP functions for forward matrix-vector products
    def jvp_dtheta_active_dtheta(v_theta):
        """
        Compute [∂Θ_active/∂θ] @ v_theta using JVP
        """
        if not np.any(v_theta):
            return np.zeros(n_lambda_theta)

        v_theta_jax = jnp.array(v_theta)

        _, jvp_result = jax.jvp(Θ_active, (Θ_opt,), (v_theta_jax,))
        return np.array(jvp_result)

    def jvp_dg_active_dtheta(v_theta):
        """
        Compute [∂g_active/∂θ] @ v_theta using JVP
        """
        if not np.any(v_theta):
            return np.zeros(n_lambda_g)

        v_theta_jax = jnp.array(v_theta)

        _, jvp_result = jax.jvp(lambda theta: g_active(theta, p_opt), (Θ_opt,), (v_theta_jax,))
        return np.array(jvp_result)

    def matvec_transpose(v):
        """
        Compute matrix-vector product for [∂R/∂u]^T @ v
        This implements the transpose of the UDE Jacobian matrix.
        All operations use JVPs instead of explicit matrix formation.
        """
        # Split v into blocks
        idx = 0
        v_p = v[idx:idx+n_p]
        idx += n_p
        v_btheta = v[idx:idx+n_btheta]
        idx += n_btheta
        v_bg = v[idx:idx+n_bg]
        idx += n_bg
        v_theta = v[idx:idx+n_theta]
        idx += n_theta
        v_lambda_theta = v[idx:idx+n_lambda_theta]
        idx += n_lambda_theta
        v_lambda_g = v[idx:idx+n_lambda_g]
        idx += n_lambda_g
        v_outputs = v[idx:idx+n_outputs_total]
        v_f = v_outputs[:n_f]
        v_f_theta = v_outputs[n_f:]

        print(v)

        # Initialize result
        result = np.zeros(total_size)

        # Block 1: Effect on p
        result[:n_p] = v_p
        result[:n_p] -= jvp_d2L_dtheta_dp_T(v_theta)
        result[:n_p] -= jvp_dg_active_dp_T(v_lambda_g)
        result[:n_p] -= jvp_df_dp_T(v_f)

        # Block 2: Effect on b_theta
        idx = n_p
        result[idx:idx+n_btheta] = v_btheta + v_lambda_theta

        # Block 3: Effect on b_g
        idx += n_btheta
        result[idx:idx+n_bg] = v_bg + v_lambda_g

        # Block 4: Effect on theta
        idx += n_bg
        result_theta = np.zeros(n_theta)
        if np.any(v_theta):
            print('using hessian-vector product!')
            # Use Hessian-vector product instead of full Hessian
            result_theta -= hessian_vector_product(v_theta)
        result_theta -= jvp_dtheta_active_dtheta_T(v_lambda_theta)
        result_theta -= jvp_dg_active_dtheta_T(v_lambda_g)
        result_theta -= jvp_df_dtheta_T(v_f)
        # Identity outputs contribution
        if np.any(v_f_theta):
            result_theta -= v_f_theta  # -I @ v_f_theta
        result[idx:idx+n_theta] = result_theta

        # Block 5: Effect on lambda_theta
        idx += n_theta
        result[idx:idx+n_lambda_theta] = -jvp_dtheta_active_dtheta(v_theta)

        # Block 6: Effect on lambda_g
        idx += n_lambda_theta
        result[idx:idx+n_lambda_g] = -jvp_dg_active_dtheta(v_theta)

        # Block 7: Effect on outputs
        idx += n_lambda_g
        result[idx:idx+n_outputs_total] = v_outputs

        return result

    # Create LinearOperator
    A_transpose = LinearOperator((total_size, total_size), matvec=matvec_transpose)

    # Storage for sensitivities
    sensitivities = {}

    # Solve for each output
    output_names = ['f', 'θ₀', 'θ₁']

    for i, name in enumerate(output_names):
        # Create RHS vector with 1 in appropriate output position
        rhs = np.zeros(total_size)
        output_start_idx = n_p + n_btheta + n_bg + n_theta + n_lambda_theta + n_lambda_g
        rhs[output_start_idx + i] = 1.0

        # Solve the system
        print(f"\nSolving for sensitivities of {name}...")
        solution, info = gmres(A_transpose, rhs, rtol=1e-10, maxiter=1000)

        if info == 0:
            # Extract sensitivities
            sens_p = solution[:n_p]
            sens_btheta = solution[n_p:n_p+n_btheta]
            sens_bg = solution[n_p+n_btheta:n_p+n_btheta+n_bg]

            sensitivities[name] = {
                'wrt_p': sens_p,
                'wrt_btheta': sens_btheta,
                'wrt_bg': sens_bg
            }

            print(f"  d{name}/dp₀ = {sens_p[0]:+.6f}")
            print(f"  d{name}/dp₁ = {sens_p[1]:+.6f}")
            print(f"  d{name}/dp₂ = {sens_p[2]:+.6f}")
            print(f"  d{name}/db_θ₀ = {sens_btheta[0]:+.6f}")
            print(f"  d{name}/db_g₀ = {sens_bg[0]:+.6f}")
        else:
            print(f"  Warning: GMRES did not converge (info={info})")

    return sensitivities

def verify_with_finite_differences(sensitivities, h=1e-6):
    """
    Verify sensitivities using finite differences.
    """
    print("\n" + "="*60)
    print("Verification with Finite Differences")
    print("="*60)

    # Base values
    Θ_base = jnp.array([6.0, -6.0])
    p_base = jnp.array([3.0, 4.0, 3.0])
    b_theta_base = 6.0

    # For simplicity, we'll verify df/dp₀ and dθ₀/db_θ₀

    # Verify df/dp₀
    def solve_optimization(p_val, b_theta_val):
        """
        Simplified optimization solve assuming active constraints remain active.
        For the active set: θ₀ = b_theta_val, θ₁ = -b_theta_val
        """
        θ_opt = jnp.array([b_theta_val, -b_theta_val])
        f_val = f(θ_opt, p_val)[0]
        return f_val, θ_opt

    # df/dp₀ by finite differences
    p_plus = p_base.at[0].set(p_base[0] + h)
    f_plus, _ = solve_optimization(p_plus, b_theta_base)
    p_minus = p_base.at[0].set(p_base[0] - h)
    f_minus, _ = solve_optimization(p_minus, b_theta_base)
    df_dp0_fd = (f_plus - f_minus) / (2 * h)

    print(f"\ndf/dp₀:")
    print(f"  UDE:    {sensitivities['f']['wrt_p'][0]:+.6f}")
    print(f"  FD:     {df_dp0_fd:+.6f}")
    print(f"  Error:  {abs(sensitivities['f']['wrt_p'][0] - df_dp0_fd):.2e}")

    # dθ₀/db_θ₀ by finite differences (should be 1.0 since θ₀ = b_θ₀ at optimum)
    _, theta_plus = solve_optimization(p_base, b_theta_base + h)
    _, theta_minus = solve_optimization(p_base, b_theta_base - h)
    dtheta0_dbtheta_fd = (theta_plus[0] - theta_minus[0]) / (2 * h)

    print(f"\ndθ₀/db_θ₀:")
    print(f"  UDE:    {sensitivities['θ₀']['wrt_btheta'][0]:+.6f}")
    print(f"  FD:     {dtheta0_dbtheta_fd:+.6f}")
    print(f"  Error:  {abs(sensitivities['θ₀']['wrt_btheta'][0] - dtheta0_dbtheta_fd):.2e}")

if __name__ == "__main__":
    print("="*60)
    print("Post-Optimality Sensitivity Analysis using JAX")
    print("with Jacobian-Vector Products for All Operations")
    print("="*60)

    # Compute sensitivities
    sensitivities = compute_sensitivities_jax()

    # Verify with finite differences
    verify_with_finite_differences(sensitivities)

    print("\n" + "="*60)
    print("Summary of Key Results")
    print("="*60)
    print("\nNote: Since θ₀ is bounded at 6.0 and the constraint θ₀ + θ₁ = 0 is active:")
    print("- Changes in b_θ₀ directly affect θ₀ (dθ₀/db_θ₀ = 1)")
    print("- Changes in b_θ₀ inversely affect θ₁ (dθ₁/db_θ₀ = -1)")
    print("- The objective is most sensitive to p₀ (df/dp₀ = -6)")
    print("\nAdvantages of Jacobian-Vector Product approach:")
    print("- Memory efficient: O(n) operations, no matrix storage")
    print("- Automatic differentiation ensures exact derivatives")
    print("- Scales linearly with problem size")
    print("- Numerically stable and accurate")

Post-Optimality Sensitivity Analysis using JAX
with Jacobian-Vector Products for All Operations
[0 0 0 0 0 0 0 0 0 0 0 0]

Solving for sensitivities of f...
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[ 0.79471941  0.52981294  0.13245324  0.          0.          0.
 -0.26490647 -0.         -0.          0.          0.          0.        ]
using hessian-vector product!
[-0.05448525  0.48128646 -0.00908087  0.          0.          0.258805
  0.79457674  0.          0.25880498  0.          0.          0.        ]
using hessian-vector product!
[ 0.03587375  0.11430935 -0.08025979  0.          0.25871623 -0.8171906
  0.29611005 -0.25871625 -0.29975822  0.          0.          0.        ]
using hessian-vector product!
[-0.20882484  0.26031975 -0.00167373 -0.35466231  0.6099331   0.17641066
 -0.10667187  0.45405355 -0.37703453  0.          0.          0.        ]
using hessian-vector product!
[-0.28802468  0.28758354  0.09178891  0.72515838  0.35945893  0.13373641
 -0.24301251 -0.30577933  0.0201340

## The All-VJP way 

In [ ]:
"""
Post-optimality sensitivity analysis using JAX and the UDE approach
with all matrix-vector products computed using Vector-Jacobian Products (VJPs).
"""

import jax
import jax.numpy as jnp
import numpy as np
from scipy.sparse.linalg import LinearOperator, gmres
from functools import partial

# Define the optimization problem functions
def f(Θ, p):
    """Objective function"""
    f_val = (Θ[0] - p[0])**2 + Θ[0] * Θ[1] + (Θ[1] + p[1])**2 - p[2]
    return jnp.array([f_val])

def Θ_active(Θ):
    """Active bound constraint on Θ[0]"""
    return jnp.array([Θ[0]])

def g_active(Θ, p):
    """Active equality constraint"""
    return jnp.array([Θ[0] + Θ[1]])

def compute_sensitivities_jax():
    """
    Compute post-optimality sensitivities using JAX and the UDE approach.
    Uses Vector-Jacobian Products (VJPs) for all matrix-transpose-vector operations.
    """

    # Known optimal solution
    Θ_opt = jnp.array([6.0, -6.0])
    p_opt = jnp.array([3.0, 4.0, 3.0])
    b_theta_opt = jnp.array([6.0])
    b_g_opt = jnp.array([0.0])
    λ_theta_opt = jnp.array([2.0])
    λ_g_opt = jnp.array([-2.0])

    # Problem dimensions
    n_p = 3  # parameters
    n_btheta = 1  # active bounds on design vars
    n_bg = 1  # active constraint bounds
    n_theta = 2  # design variables
    n_lambda_theta = 1  # multipliers for active bounds
    n_lambda_g = 1  # multipliers for active constraints
    n_f = 1  # original objective
    n_f_theta = 2  # identity outputs for design variables
    n_outputs_total = n_f + n_f_theta  # total outputs

    total_size = n_p + n_btheta + n_bg + n_theta + n_lambda_theta + n_lambda_g + n_outputs_total

    # Define Lagrangian and its gradient
    def lagrangian(Θ, p, λ_theta, λ_g):
        """Lagrangian function"""
        return f(Θ, p)[0] + λ_theta @ Θ_active(Θ) + λ_g @ g_active(Θ, p)

    def lagrangian_grad(Θ, p, λ_theta, λ_g):
        """Gradient of Lagrangian w.r.t. Θ"""
        return jax.grad(lagrangian, argnums=0)(Θ, p, λ_theta, λ_g)

    # Function to compute Hessian-vector product using finite differences
    def hessian_vector_product(v_theta, h=1e-8):
        """
        Compute H @ v where H is the Hessian of the Lagrangian w.r.t. Θ
        Uses finite differences: H @ v ≈ (∇L(Θ + h*v) - ∇L(Θ - h*v)) / (2*h)
        """
        if not np.any(v_theta):
            return np.zeros(n_theta)

        v_theta_jax = jnp.array(v_theta)

        # Compute gradients at perturbed points
        grad_plus = lagrangian_grad(Θ_opt + h * v_theta_jax, p_opt, λ_theta_opt, λ_g_opt)
        grad_minus = lagrangian_grad(Θ_opt - h * v_theta_jax, p_opt, λ_theta_opt, λ_g_opt)

        # Finite difference approximation of Hessian-vector product
        hvp = (grad_plus - grad_minus) / (2 * h)

        return np.array(hvp)

    # VJP functions for matrix-transpose-vector products
    def vjp_d2L_dtheta_dp_T(v_theta):
        """
        Compute [∂²L/∂θ∂p]^T @ v_theta using VJP
        """
        if not np.any(v_theta):
            return np.zeros(n_p)

        v_theta_jax = jnp.array(v_theta)

        # Get VJP function for the gradient w.r.t. θ
        _, vjp_fun = jax.vjp(lambda p: lagrangian_grad(Θ_opt, p, λ_theta_opt, λ_g_opt), p_opt)

        # Apply VJP with v_theta
        result = vjp_fun(v_theta_jax)[0]
        return np.array(result)

    def vjp_dg_active_dp_T(v_lambda_g):
        """
        Compute [∂g_active/∂p]^T @ v_lambda_g using VJP
        """
        if not np.any(v_lambda_g):
            return np.zeros(n_p)

        v_lambda_g_jax = jnp.array(v_lambda_g)

        # Get VJP function for g_active w.r.t. p
        _, vjp_fun = jax.vjp(lambda p: g_active(Θ_opt, p), p_opt)

        # Apply VJP with v_lambda_g
        result = vjp_fun(v_lambda_g_jax)[0]
        return np.array(result)

    def vjp_df_dp_T(v_f):
        """
        Compute [∂f/∂p]^T @ v_f using VJP
        """
        if not np.any(v_f):
            return np.zeros(n_p)

        v_f_jax = jnp.array(v_f)

        # Get VJP function for f w.r.t. p
        _, vjp_fun = jax.vjp(lambda p: f(Θ_opt, p), p_opt)

        # Apply VJP with v_f
        result = vjp_fun(v_f_jax)[0]
        return np.array(result)

    def vjp_dtheta_active_dtheta_T(v_lambda_theta):
        """
        Compute [∂Θ_active/∂θ]^T @ v_lambda_theta using VJP
        """
        if not np.any(v_lambda_theta):
            return np.zeros(n_theta)

        v_lambda_theta_jax = jnp.array(v_lambda_theta)

        # Get VJP function for Θ_active w.r.t. θ
        _, vjp_fun = jax.vjp(Θ_active, Θ_opt)

        # Apply VJP with v_lambda_theta
        result = vjp_fun(v_lambda_theta_jax)[0]
        return np.array(result)

    def vjp_dg_active_dtheta_T(v_lambda_g):
        """
        Compute [∂g_active/∂θ]^T @ v_lambda_g using VJP
        """
        if not np.any(v_lambda_g):
            return np.zeros(n_theta)

        v_lambda_g_jax = jnp.array(v_lambda_g)

        # Get VJP function for g_active w.r.t. θ
        _, vjp_fun = jax.vjp(lambda theta: g_active(theta, p_opt), Θ_opt)

        # Apply VJP with v_lambda_g
        result = vjp_fun(v_lambda_g_jax)[0]
        return np.array(result)

    def vjp_df_dtheta_T(v_f):
        """
        Compute [∂f/∂θ]^T @ v_f using VJP
        """
        if not np.any(v_f):
            return np.zeros(n_theta)

        v_f_jax = jnp.array(v_f)

        # Get VJP function for f w.r.t. θ
        _, vjp_fun = jax.vjp(lambda theta: f(theta, p_opt), Θ_opt)

        # Apply VJP with v_f
        result = vjp_fun(v_f_jax)[0]
        return np.array(result)

    # Regular JVP functions for forward matrix-vector products
    def jvp_dtheta_active_dtheta(v_theta):
        """
        Compute [∂Θ_active/∂θ] @ v_theta using JVP
        """
        if not np.any(v_theta):
            return np.zeros(n_lambda_theta)

        v_theta_jax = jnp.array(v_theta)

        _, jvp_result = jax.jvp(Θ_active, (Θ_opt,), (v_theta_jax,))
        return np.array(jvp_result)

    def jvp_dg_active_dtheta(v_theta):
        """
        Compute [∂g_active/∂θ] @ v_theta using JVP
        """
        if not np.any(v_theta):
            return np.zeros(n_lambda_g)

        v_theta_jax = jnp.array(v_theta)

        _, jvp_result = jax.jvp(lambda theta: g_active(theta, p_opt), (Θ_opt,), (v_theta_jax,))
        return np.array(jvp_result)

    def matvec_transpose(v):
        """
        Compute matrix-vector product for [∂R/∂u]^T @ v
        This implements the transpose of the UDE Jacobian matrix.
        Uses VJPs for transpose operations and JVPs for forward operations.
        """
        # Split v into blocks
        idx = 0
        v_p = v[idx:idx+n_p]
        idx += n_p
        v_btheta = v[idx:idx+n_btheta]
        idx += n_btheta
        v_bg = v[idx:idx+n_bg]
        idx += n_bg
        v_theta = v[idx:idx+n_theta]
        idx += n_theta
        v_lambda_theta = v[idx:idx+n_lambda_theta]
        idx += n_lambda_theta
        v_lambda_g = v[idx:idx+n_lambda_g]
        idx += n_lambda_g
        v_outputs = v[idx:idx+n_outputs_total]
        v_f = v_outputs[:n_f]
        v_f_theta = v_outputs[n_f:]

        print(v)

        # Initialize result
        result = np.zeros(total_size)

        # Block 1: Effect on p
        result[:n_p] = v_p
        result[:n_p] -= vjp_d2L_dtheta_dp_T(v_theta)
        result[:n_p] -= vjp_dg_active_dp_T(v_lambda_g)
        result[:n_p] -= vjp_df_dp_T(v_f)

        # Block 2: Effect on b_theta
        idx = n_p
        result[idx:idx+n_btheta] = v_btheta + v_lambda_theta

        # Block 3: Effect on b_g
        idx += n_btheta
        result[idx:idx+n_bg] = v_bg + v_lambda_g

        # Block 4: Effect on theta
        idx += n_bg
        result_theta = np.zeros(n_theta)
        if np.any(v_theta):
            print('using hessian-vector product!')
            # Use Hessian-vector product for second derivatives
            result_theta -= hessian_vector_product(v_theta)
        result_theta -= vjp_dtheta_active_dtheta_T(v_lambda_theta)
        result_theta -= vjp_dg_active_dtheta_T(v_lambda_g)
        result_theta -= vjp_df_dtheta_T(v_f)
        # Identity outputs contribution
        if np.any(v_f_theta):
            result_theta -= v_f_theta  # -I @ v_f_theta
        result[idx:idx+n_theta] = result_theta

        # Block 5: Effect on lambda_theta
        idx += n_theta
        result[idx:idx+n_lambda_theta] = -jvp_dtheta_active_dtheta(v_theta)

        # Block 6: Effect on lambda_g
        idx += n_lambda_theta
        result[idx:idx+n_lambda_g] = -jvp_dg_active_dtheta(v_theta)

        # Block 7: Effect on outputs
        idx += n_lambda_g
        result[idx:idx+n_outputs_total] = v_outputs

        return result

    # Create LinearOperator
    A_transpose = LinearOperator((total_size, total_size), matvec=matvec_transpose)

    # Storage for sensitivities
    sensitivities = {}

    # Solve for each output
    output_names = ['f', 'θ₀', 'θ₁']

    for i, name in enumerate(output_names):
        # Create RHS vector with 1 in appropriate output position
        rhs = np.zeros(total_size)
        output_start_idx = n_p + n_btheta + n_bg + n_theta + n_lambda_theta + n_lambda_g
        rhs[output_start_idx + i] = 1.0

        # Solve the system
        print(f"\nSolving for sensitivities of {name}...")
        solution, info = gmres(A_transpose, rhs, rtol=1e-10, maxiter=1000)

        if info == 0:
            # Extract sensitivities
            sens_p = solution[:n_p]
            sens_btheta = solution[n_p:n_p+n_btheta]
            sens_bg = solution[n_p+n_btheta:n_p+n_btheta+n_bg]

            sensitivities[name] = {
                'wrt_p': sens_p,
                'wrt_btheta': sens_btheta,
                'wrt_bg': sens_bg
            }

            print(f"  d{name}/dp₀ = {sens_p[0]:+.6f}")
            print(f"  d{name}/dp₁ = {sens_p[1]:+.6f}")
            print(f"  d{name}/dp₂ = {sens_p[2]:+.6f}")
            print(f"  d{name}/db_θ₀ = {sens_btheta[0]:+.6f}")
            print(f"  d{name}/db_g₀ = {sens_bg[0]:+.6f}")
        else:
            print(f"  Warning: GMRES did not converge (info={info})")

    return sensitivities

def verify_with_finite_differences(sensitivities, h=1e-6):
    """
    Verify sensitivities using finite differences.
    """
    print("\n" + "="*60)
    print("Verification with Finite Differences")
    print("="*60)

    # Base values
    Θ_base = jnp.array([6.0, -6.0])
    p_base = jnp.array([3.0, 4.0, 3.0])
    b_theta_base = 6.0

    # For simplicity, we'll verify df/dp₀ and dθ₀/db_θ₀

    # Verify df/dp₀
    def solve_optimization(p_val, b_theta_val):
        """
        Simplified optimization solve assuming active constraints remain active.
        For the active set: θ₀ = b_theta_val, θ₁ = -b_theta_val
        """
        θ_opt = jnp.array([b_theta_val, -b_theta_val])
        f_val = f(θ_opt, p_val)[0]
        return f_val, θ_opt

    # df/dp₀ by finite differences
    p_plus = p_base.at[0].set(p_base[0] + h)
    f_plus, _ = solve_optimization(p_plus, b_theta_base)
    p_minus = p_base.at[0].set(p_base[0] - h)
    f_minus, _ = solve_optimization(p_minus, b_theta_base)
    df_dp0_fd = (f_plus - f_minus) / (2 * h)

    print(f"\ndf/dp₀:")
    print(f"  UDE:    {sensitivities['f']['wrt_p'][0]:+.6f}")
    print(f"  FD:     {df_dp0_fd:+.6f}")
    print(f"  Error:  {abs(sensitivities['f']['wrt_p'][0] - df_dp0_fd):.2e}")

    # dθ₀/db_θ₀ by finite differences (should be 1.0 since θ₀ = b_θ₀ at optimum)
    _, theta_plus = solve_optimization(p_base, b_theta_base + h)
    _, theta_minus = solve_optimization(p_base, b_theta_base - h)
    dtheta0_dbtheta_fd = (theta_plus[0] - theta_minus[0]) / (2 * h)

    print(f"\ndθ₀/db_θ₀:")
    print(f"  UDE:    {sensitivities['θ₀']['wrt_btheta'][0]:+.6f}")
    print(f"  FD:     {dtheta0_dbtheta_fd:+.6f}")
    print(f"  Error:  {abs(sensitivities['θ₀']['wrt_btheta'][0] - dtheta0_dbtheta_fd):.2e}")

if __name__ == "__main__":
    print("="*60)
    print("Post-Optimality Sensitivity Analysis using JAX")
    print("with Vector-Jacobian Products (VJPs)")
    print("="*60)

    # Compute sensitivities
    sensitivities = compute_sensitivities_jax()

    # Verify with finite differences
    verify_with_finite_differences(sensitivities)

    print("\n" + "="*60)
    print("Summary of Key Results")
    print("="*60)
    print("\nNote: Since θ₀ is bounded at 6.0 and the constraint θ₀ + θ₁ = 0 is active:")
    print("- Changes in b_θ₀ directly affect θ₀ (dθ₀/db_θ₀ = 1)")
    print("- Changes in b_θ₀ inversely affect θ₁ (dθ₁/db_θ₀ = -1)")
    print("- The objective is most sensitive to p₀ (df/dp₀ = -6)")
    print("\nAdvantages of Vector-Jacobian Product (VJP) approach:")
    print("- Memory efficient: O(n) operations, no matrix storage")
    print("- VJPs are the natural choice for A^T @ v operations")
    print("- Automatic differentiation ensures exact derivatives")
    print("- Scales linearly with problem size")
    print("- More efficient than the JVP approach for transpose operations")

Post-Optimality Sensitivity Analysis using JAX
with Vector-Jacobian Products (VJPs)
[0 0 0 0 0 0 0 0 0 0 0 0]

Solving for sensitivities of f...
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[ 0.79471941  0.52981294  0.13245324  0.          0.          0.
 -0.26490647 -0.         -0.          0.          0.          0.        ]
using hessian-vector product!
[-0.05448525  0.48128646 -0.00908087  0.          0.          0.258805
  0.79457674  0.          0.25880498  0.          0.          0.        ]
using hessian-vector product!
[ 0.03587375  0.11430935 -0.08025979  0.          0.25871623 -0.8171906
  0.29611005 -0.25871625 -0.29975822  0.          0.          0.        ]
using hessian-vector product!
[-0.20882484  0.26031975 -0.00167373 -0.35466231  0.6099331   0.17641066
 -0.10667187  0.45405355 -0.37703453  0.          0.          0.        ]
using hessian-vector product!
[-0.28802468  0.28758354  0.09178891  0.72515838  0.35945893  0.13373641
 -0.24301251 -0.30577933  0.02013409  0.       